# Chapter 20: Profiling and Timing Python Code

Before optimizing code, you must **measure** it. This notebook covers Python's built-in
tools for timing code snippets, profiling function calls, and identifying performance
bottlenecks.

## Topics Covered
- **timeit**: Precise micro-benchmarking of small code snippets
- **cProfile**: Function-level profiling with call counts and timing
- **pstats**: Analyzing and sorting profile results
- **time.perf_counter()**: Manual high-resolution timing
- **Decorator-based profiling**: Reusable profiling patterns
- **Common performance traps**: String concatenation, repeated lookups, and more

## The timeit Module: Micro-Benchmarking

`timeit` runs a code snippet many times and reports the total (or average) execution time.
It automatically disables garbage collection during timing and uses the best available timer
for your platform, making it ideal for comparing small code alternatives.

In [ ]:
import timeit

# timeit.timeit() runs the statement `number` times and returns total seconds
# Default number is 1_000_000

# Compare list creation approaches
time_literal = timeit.timeit("[1, 2, 3]", number=1_000_000)
time_constructor = timeit.timeit("list((1, 2, 3))", number=1_000_000)

print("Creating [1, 2, 3] one million times:")
print(f"  List literal:      {time_literal:.4f}s")
print(f"  list() constructor: {time_constructor:.4f}s")
print(f"  Literal is {time_constructor / time_literal:.1f}x faster")

# Compare dict creation
time_dict_literal = timeit.timeit('{"a": 1, "b": 2}', number=1_000_000)
time_dict_constructor = timeit.timeit('dict(a=1, b=2)', number=1_000_000)

print(f"\nCreating dict one million times:")
print(f"  Dict literal: {time_dict_literal:.4f}s")
print(f"  dict():       {time_dict_constructor:.4f}s")

In [ ]:
import timeit

# timeit.repeat() runs multiple trials and returns a list of times
# This helps identify variance in measurements

results_comp = timeit.repeat(
    "[x**2 for x in range(100)]",
    repeat=5,
    number=10_000,
)

results_map = timeit.repeat(
    "list(map(lambda x: x**2, range(100)))",
    repeat=5,
    number=10_000,
)

print("Squaring 0-99, 10k iterations, 5 trials:")
print(f"  List comprehension: {[f'{t:.4f}' for t in results_comp]}")
print(f"  map() + lambda:     {[f'{t:.4f}' for t in results_map]}")
print(f"\n  Best comprehension: {min(results_comp):.4f}s")
print(f"  Best map+lambda:    {min(results_map):.4f}s")
print("\nNote: use min() of repeat results, not average (min reflects best case)")

In [ ]:
import timeit

# Using setup parameter for imports and initialization
# The setup code runs once, then the statement is timed

setup = "from collections import deque; data = list(range(1000))"

time_list_pop = timeit.timeit(
    "data.pop(0)",
    setup="data = list(range(1000))",
    number=1_000,
)

time_deque_pop = timeit.timeit(
    "d.popleft()",
    setup="from collections import deque; d = deque(range(1000))",
    number=1_000,
)

print("Popping from the left, 1000 times:")
print(f"  list.pop(0):    {time_list_pop:.6f}s")
print(f"  deque.popleft(): {time_deque_pop:.6f}s")
print(f"  deque is {time_list_pop / time_deque_pop:.1f}x faster")

# Timing a callable directly with timeit.timeit()
def fibonacci(n: int) -> int:
    """Naive recursive fibonacci."""
    if n < 2:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

time_fib = timeit.timeit(lambda: fibonacci(20), number=100)
print(f"\nfibonacci(20) x 100: {time_fib:.4f}s")
print(f"  Average per call:  {time_fib / 100 * 1000:.2f}ms")

## cProfile: Function-Level Profiling

`cProfile` is a deterministic profiler that records every function call, how many times it was
called, and how long each call took. It reports both:
- **tottime**: Time spent *inside* the function (excluding sub-calls)
- **cumtime**: Cumulative time including all sub-calls

This is the go-to tool for identifying which functions consume the most time.

In [ ]:
import cProfile


def process_data(n: int) -> list[int]:
    """Simulate a multi-step data pipeline."""
    raw = generate_data(n)
    filtered = filter_data(raw)
    result = transform_data(filtered)
    return result


def generate_data(n: int) -> list[int]:
    """Generate a list of numbers."""
    return [i * 3 for i in range(n)]


def filter_data(data: list[int]) -> list[int]:
    """Filter to even numbers only."""
    return [x for x in data if x % 2 == 0]


def transform_data(data: list[int]) -> list[int]:
    """Apply an expensive transformation."""
    result: list[int] = []
    for x in data:
        result.append(slow_computation(x))
    return result


def slow_computation(x: int) -> int:
    """Simulate an expensive per-element computation."""
    total = 0
    for i in range(100):
        total += x * i
    return total


# Profile the pipeline
print("Profiling process_data(5000):")
print("=" * 70)
cProfile.run("process_data(5000)", sort="cumulative")

## pstats: Analyzing Profile Results

For more control over profile output, save the results to a `pstats.Stats` object.
You can sort by different columns, filter by function name, and strip directory paths
for cleaner output.

In [ ]:
import cProfile
import pstats
import io


def compute_primes(limit: int) -> list[int]:
    """Find primes up to limit using trial division."""
    primes: list[int] = []
    for num in range(2, limit):
        if is_prime(num):
            primes.append(num)
    return primes


def is_prime(n: int) -> bool:
    """Check if n is prime via trial division."""
    if n < 2:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True


# Profile into a Stats object for analysis
profiler = cProfile.Profile()
profiler.enable()
result = compute_primes(5000)
profiler.disable()

print(f"Found {len(result)} primes under 5000")
print()

# Analyze with pstats
stream = io.StringIO()
stats = pstats.Stats(profiler, stream=stream)
stats.strip_dirs()           # Remove directory paths for readability
stats.sort_stats("tottime")  # Sort by time spent in function
stats.print_stats(10)        # Top 10 functions
print(stream.getvalue())

In [ ]:
import cProfile
import pstats
import io


def run_simulation(iterations: int) -> float:
    """Run a multi-step simulation."""
    total: float = 0.0
    for i in range(iterations):
        value = step_one(i)
        value = step_two(value)
        total += step_three(value)
    return total


def step_one(x: int) -> float:
    """First processing step."""
    return sum(i * 0.1 for i in range(x % 50))


def step_two(x: float) -> float:
    """Second processing step (deliberately slow)."""
    result = x
    for _ in range(200):
        result = (result + 1.0) * 0.999
    return result


def step_three(x: float) -> float:
    """Third processing step."""
    return x ** 0.5


# Identifying bottlenecks: sort by cumtime vs tottime
profiler = cProfile.Profile()
profiler.enable()
run_simulation(500)
profiler.disable()

# Sort by cumulative time (includes sub-calls)
stream = io.StringIO()
stats = pstats.Stats(profiler, stream=stream)
stats.strip_dirs()
print("Sorted by CUMULATIVE time (cumtime):")
stats.sort_stats("cumulative")
stats.print_stats(8)
print(stream.getvalue())

# Sort by total time in function (excludes sub-calls)
stream2 = io.StringIO()
stats2 = pstats.Stats(profiler, stream=stream2)
stats2.strip_dirs()
print("\nSorted by TOTAL time in function (tottime):")
stats2.sort_stats("tottime")
stats2.print_stats(8)
print(stream2.getvalue())

## time.perf_counter(): Manual High-Resolution Timing

When you need to time a specific section of code (not just a function call), use
`time.perf_counter()`. It provides the highest resolution clock available on your system
and includes time spent sleeping. For measuring only CPU time, use `time.process_time()`.

In [ ]:
import time

# time.perf_counter() for wall-clock time
start = time.perf_counter()
total = sum(i * i for i in range(1_000_000))
end = time.perf_counter()
print(f"Sum of squares (1M): {end - start:.4f}s")

# time.perf_counter_ns() for nanosecond precision
start_ns = time.perf_counter_ns()
result = sorted([5, 3, 1, 4, 2])
end_ns = time.perf_counter_ns()
print(f"Sorting 5 elements: {end_ns - start_ns} ns")

# time.process_time() measures only CPU time (not sleep/IO)
cpu_start = time.process_time()
data = [x ** 2 for x in range(500_000)]
cpu_end = time.process_time()
print(f"\nCPU time for 500k squares: {cpu_end - cpu_start:.4f}s")

# Demonstrate the difference: perf_counter includes sleep, process_time does not
wall_start = time.perf_counter()
cpu_start = time.process_time()
time.sleep(0.1)  # Sleep 100ms
_ = sum(range(100_000))
wall_end = time.perf_counter()
cpu_end = time.process_time()

print(f"\nWith 100ms sleep + computation:")
print(f"  Wall clock (perf_counter): {wall_end - wall_start:.4f}s")
print(f"  CPU time (process_time):   {cpu_end - cpu_start:.4f}s")

## Profiling Patterns: Decorator-Based Profiling

A profiling decorator makes it easy to measure any function without modifying its code.
This is a reusable pattern you can drop into any project.

In [ ]:
import functools
import time
from typing import Any, Callable, TypeVar

F = TypeVar("F", bound=Callable[..., Any])


def timer(func: F) -> F:
    """Decorator that prints execution time of the wrapped function."""
    @functools.wraps(func)
    def wrapper(*args: Any, **kwargs: Any) -> Any:
        start = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - start
        print(f"  {func.__name__}() took {elapsed:.4f}s")
        return result
    return wrapper  # type: ignore[return-value]


def call_counter(func: F) -> F:
    """Decorator that counts how many times a function is called."""
    @functools.wraps(func)
    def wrapper(*args: Any, **kwargs: Any) -> Any:
        wrapper.call_count += 1  # type: ignore[attr-defined]
        return func(*args, **kwargs)
    wrapper.call_count = 0  # type: ignore[attr-defined]
    return wrapper  # type: ignore[return-value]


# Using the timer decorator
@timer
def slow_sort(data: list[int]) -> list[int]:
    """Bubble sort - deliberately slow for demonstration."""
    arr = data.copy()
    n = len(arr)
    for i in range(n):
        for j in range(0, n - i - 1):
            if arr[j] > arr[j + 1]:
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
    return arr


@timer
def fast_sort(data: list[int]) -> list[int]:
    """Built-in sort (Timsort)."""
    return sorted(data)


import random
test_data = [random.randint(0, 10_000) for _ in range(5_000)]

print("Sorting 5,000 random integers:")
result1 = slow_sort(test_data)
result2 = fast_sort(test_data)
print(f"  Results match: {result1 == result2}")

In [ ]:
import functools
import time
from typing import Any, Callable


def profile_stats(func: Callable[..., Any]) -> Callable[..., Any]:
    """Decorator that accumulates timing statistics across multiple calls."""
    @functools.wraps(func)
    def wrapper(*args: Any, **kwargs: Any) -> Any:
        start = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - start
        wrapper._times.append(elapsed)  # type: ignore[attr-defined]
        return result

    wrapper._times: list[float] = []  # type: ignore[attr-defined]

    def get_stats() -> dict[str, float]:
        times = wrapper._times  # type: ignore[attr-defined]
        if not times:
            return {"calls": 0}
        return {
            "calls": len(times),
            "total": sum(times),
            "avg": sum(times) / len(times),
            "min": min(times),
            "max": max(times),
        }

    wrapper.get_stats = get_stats  # type: ignore[attr-defined]
    return wrapper


@profile_stats
def process_item(x: int) -> int:
    """Simulate variable-cost processing."""
    return sum(range(x))


# Run the function many times with varying input sizes
for i in range(100):
    process_item(i * 100)

# Inspect accumulated statistics
stats = process_item.get_stats()  # type: ignore[attr-defined]
print("Profile statistics for process_item():")
for key, value in stats.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.6f}s")
    else:
        print(f"  {key}: {value}")

## Common Performance Traps

Certain coding patterns are deceptively slow. Measuring them with `timeit` makes the
cost obvious and helps you choose better alternatives.

In [ ]:
import timeit

# Trap 1: String concatenation in a loop
# Strings are immutable, so += creates a new string each time: O(n^2)

def concat_with_plus(n: int) -> str:
    """Build a string using += (slow)."""
    result = ""
    for i in range(n):
        result += str(i) + " "
    return result


def concat_with_join(n: int) -> str:
    """Build a string using join (fast)."""
    return " ".join(str(i) for i in range(n))


def concat_with_list(n: int) -> str:
    """Build a string by appending to a list, then joining."""
    parts: list[str] = []
    for i in range(n):
        parts.append(str(i))
    return " ".join(parts)


n = 10_000
t_plus = timeit.timeit(lambda: concat_with_plus(n), number=10)
t_join = timeit.timeit(lambda: concat_with_join(n), number=10)
t_list = timeit.timeit(lambda: concat_with_list(n), number=10)

print(f"String concatenation ({n} items, 10 runs):")
print(f"  += loop:        {t_plus:.4f}s")
print(f"  join(genexpr):  {t_join:.4f}s")
print(f"  list + join:    {t_list:.4f}s")

In [ ]:
import timeit

# Trap 2: Repeated attribute/method lookups in tight loops
# Each dot lookup (e.g., math.sqrt) is a dictionary lookup

setup = "import math; data = list(range(1, 10001))"

# Slow: repeated global + attribute lookup
slow_code = """
result = []
for x in data:
    result.append(math.sqrt(x))
"""

# Fast: cache the method lookup and the append method
fast_code = """
sqrt = math.sqrt
result = []
append = result.append
for x in data:
    append(sqrt(x))
"""

# Fastest: use a list comprehension (implicit optimization)
fastest_code = "result = [math.sqrt(x) for x in data]"

t_slow = timeit.timeit(slow_code, setup=setup, number=1000)
t_fast = timeit.timeit(fast_code, setup=setup, number=1000)
t_fastest = timeit.timeit(fastest_code, setup=setup, number=1000)

print("sqrt() over 10k items, 1000 runs:")
print(f"  Repeated lookups:   {t_slow:.4f}s")
print(f"  Cached lookups:     {t_fast:.4f}s")
print(f"  List comprehension: {t_fastest:.4f}s")

In [ ]:
import timeit

# Trap 3: Checking membership in a list vs a set
# list membership is O(n), set membership is O(1)

setup_list = "data = list(range(10000))"
setup_set = "data = set(range(10000))"

# Worst case: item not found (must scan entire list)
t_list = timeit.timeit("9999 in data", setup=setup_list, number=100_000)
t_set = timeit.timeit("9999 in data", setup=setup_set, number=100_000)

print("Membership test (item at end), 100k checks:")
print(f"  list: {t_list:.4f}s")
print(f"  set:  {t_set:.4f}s")
print(f"  set is {t_list / t_set:.0f}x faster")

# Trap 4: Creating objects unnecessarily in loops
t_create = timeit.timeit(
    "[tuple(range(10)) for _ in range(1000)]",
    number=1000,
)
t_reuse = timeit.timeit(
    "[t for _ in range(1000)]",
    setup="t = tuple(range(10))",
    number=1000,
)

print(f"\nCreating vs reusing a tuple, 1000x1000:")
print(f"  Create each time: {t_create:.4f}s")
print(f"  Reuse one object: {t_reuse:.4f}s")

## Summary

### Key Takeaways

| Tool | Use Case | Key Function |
|------|----------|-------------|
| **timeit** | Micro-benchmark small snippets | `timeit.timeit()`, `timeit.repeat()` |
| **cProfile** | Profile all function calls | `cProfile.run()`, `cProfile.Profile()` |
| **pstats** | Analyze profile results | `Stats.sort_stats()`, `Stats.print_stats()` |
| **time.perf_counter()** | Manual wall-clock timing | High-resolution, includes sleep |
| **time.process_time()** | CPU-only timing | Excludes sleep and I/O wait |

### Profiling Workflow
1. **Measure first** -- never guess where bottlenecks are
2. **Use `cProfile`** to find which functions take the most time
3. **Sort by `cumtime`** to find the "big picture" bottlenecks
4. **Sort by `tottime`** to find where CPU time is actually spent
5. **Use `timeit`** to compare alternative implementations
6. **Re-measure** after each change to verify improvement

### Common Traps to Avoid
- String concatenation with `+=` in loops (use `"".join()` instead)
- Repeated attribute lookups in tight loops (cache with local variables)
- Membership testing in `list` when `set` would suffice
- Creating identical objects repeatedly instead of reusing them